In [2]:
import pandas as pd
import numpy as np
import string, re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [3]:
# Load dataset
df = pd.read_csv('/content/spam.csv', encoding='latin-1')
df = df[['v1', 'v2']].rename(columns={'v1': 'label', 'v2': 'message'})
df['label'] = df['label'].map({'ham': 0, 'spam': 1})


In [6]:
# Clean function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()
    return text

df['cleaned'] = df['message'].apply(clean_text)

In [7]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned'], df['label'], test_size=0.2, random_state=42
)

In [9]:
# Build pipelines
pipelines = {
    'NaiveBayes–TFIDF': Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.9)),
        ('clf', MultinomialNB())
    ]),
    'LogisticRegression–TFIDF': Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', max_df=0.9)),
        ('clf', LogisticRegression(max_iter=1000))
    ])
}

In [15]:
# Train & evaluate each
for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    print(f"\n{name} results:")
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
    print("Precision:", metrics.precision_score(y_test, y_pred))
    print("Recall:", metrics.recall_score(y_test, y_pred))
    print("F1 Score:", metrics.f1_score(y_test, y_pred))
    print("Confusion Matrix:\n", metrics.confusion_matrix(y_test, y_pred))

# Predict function
def predict_message(msg, model):
    msg_clean = clean_text(msg)
    pred = model.predict([msg_clean])[0]
    return "🚨 Spam" if pred == 1 else "✅ Not Spam"

# Example usage:
sample = pipelines['NaiveBayes–TFIDF']
print(predict_message("Congratulations! You've won a free lottery. Claim now!", sample))
print(predict_message("Hey, are we still meeting for coffee today?", sample))


NaiveBayes–TFIDF results:
Accuracy: 0.968609865470852
Precision: 1.0
Recall: 0.7666666666666667
F1 Score: 0.8679245283018868
Confusion Matrix:
 [[965   0]
 [ 35 115]]

LogisticRegression–TFIDF results:
Accuracy: 0.9506726457399103
Precision: 0.9702970297029703
Recall: 0.6533333333333333
F1 Score: 0.7808764940239044
Confusion Matrix:
 [[962   3]
 [ 52  98]]
🚨 Spam
✅ Not Spam
